# Implementation of GAT

Vincent Wilmet \
rmbr to use GPU

### init

In [1]:
import torch
print(torch.__version__)

1.10.0+cu111


In [3]:
# Installing Pytorch Geometric  
# literally had to write a stack overflow post bc
# the method of monkeys online wasnt working
# https://stackoverflow.com/a/71764992/17405760
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install ogb
!pip install umap-learn
!pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html

Found existing installation: torch-scatter 2.0.9
Uninstalling torch-scatter-2.0.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/test/*
    /usr/local/lib/python3.7/dist-packages/torch_scatter-2.0.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch_scatter/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/test/test_add.py
    /usr/local/lib/python3.7/dist-packages/test/test_cat.py
    /usr/local/lib/python3.7/dist-packages/test/test_coalesce.py
    /usr/local/lib/python3.7/dist-packages/test/test_convert.py
    /usr/local/lib/python3.7/dist-packages/test/test_diag.py
    /usr/local/lib/python3.7/dist-packages/test/test_ego_sample.py
    /usr/local/lib/python3.7/dist-packages/test/test_eye.py
    /usr/local/lib/python3.7/dist-packages/test/test_fps.py
    /usr/local/lib/python3.7/dist-packages/test/test_graclus.py
    /usr/local/lib/python3.7/dist-packages/test/test_grid.py
    /usr/local/lib/python3.7/dist-packages/test/t

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
# example colab notebooks on the pytorch geometric official website https://pytorch-geometric.readthedocs.io/en/latest/notes/colabs.html
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborSampler
from torch_geometric.nn import SAGEConv
import os.path as osp
import pandas as pd
import numpy as np
import collections
from pandas.core.common import flatten

### Dataset
<b>Dataset</b>: We use [obgn-products](https://ogb.stanford.edu/docs/nodeprop/#ogbn-products) dataset which is an undirected and unweighted graph, representing an Amazon product co-purchasing network. Nodes represent products sold in Amazon, and edges between two products indicate that the products are purchased together. Node features represent bag-of-words features taken from the product descriptions. The goal is to predict the category of a product in a multi-class classification setup, where the 47 top-level categories are used for target labels making it a <b>Node Classification Task</b>. 

In [5]:
# importing obg datatset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from pandas.core.common import flatten
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(16.7,8.27)})
sns.set_theme(style="ticks")
import collections
from scipy.special import softmax
# import umap

In [6]:
# download and loading the obg dataset
fp = '/content/drive/MyDrive/MLNS/final_project/'
dataset = PygNodePropPredDataset('ogbn-products', fp)
# root = osp.join(osp.dirname(osp.realpath('./')), 'data', 'products')
# dataset = PygNodePropPredDataset('ogbn-products', root) if u prefer to do local

In [7]:
# split_idx contains a dictionary of train, validation and test node indices
split_idx = dataset.get_idx_split()
# predefined ogb evaluator method used for validation of predictions
evaluator = Evaluator(name='ogbn-products')

In [8]:
# loading the dataset
data = dataset[0]

### EDA

In [9]:
# lets check the node ids distribution of train, test and val
print('Number of training nodes:', split_idx['train'].size(0))
print('Number of validation nodes:', split_idx['valid'].size(0))
print('Number of test nodes:', split_idx['test'].size(0))

Number of training nodes: 196615
Number of validation nodes: 39323
Number of test nodes: 2213091


In [10]:
# basic graph statistics of ogb-product graph
print("Number of nodes in the graph:", data.num_nodes)
print("Number of edges in the graph:", data.num_edges)
print("Node feature matrix with shape:", data.x.shape) # [num_nodes, num_node_features]
print("Graph connectivity in COO format with shape:", data.edge_index.shape) # [2, num_edges]
print("Target to train against :", data.y.shape) 
print("Node feature length", dataset.num_features)

Number of nodes in the graph: 2449029
Number of edges in the graph: 123718280
Node feature matrix with shape: torch.Size([2449029, 100])
Graph connectivity in COO format with shape: torch.Size([2, 123718280])
Target to train against : torch.Size([2449029, 1])
Node feature length 100


In [11]:
# checking the number of unique labels
# there are indeed 47 unique categories of product, as expected @asror
data.y.unique()

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])

### train_loader
run everything but test_loader cell if you want to retrain the model 

In [12]:
# ram go brrrrrrrrrrrrrrrrrrr
# comment this out this if ur gonna do test_loader
train_idx = split_idx['train']
train_loader = NeighborSampler(data.edge_index, node_idx=train_idx,
                               sizes=[15, 10, 5], batch_size=512,
                               shuffle=True)

KeyboardInterrupt: ignored

###test_loader
run everything except 


*   train_loader
*   train
*   saving



In [ ]:
#idk how much ram u have but mine got capped...
# u might have to restart the runtime and run everything except 
# the train_loader cell
test_loader = NeighborSampler(data.edge_index, node_idx=None, sizes=[-1],
                                  batch_size=1024, shuffle=False,
                                  num_workers=12)

### Loading

In [ ]:
# load integer to real product category from label mapping provided inside the dataset
df = pd.read_csv('/content/drive/MyDrive/MLNS/final_project/ogbn_products/mapping/labelidx2productcategory.csv.gz')

In [ ]:
# creating a dictionary of product category and corresponding integer label
label_idx, prod_cat = df.iloc[: ,0].values, df.iloc[: ,1].values
label_mapping = dict(zip(label_idx, prod_cat))
print("Label mapping", label_mapping)

In [ ]:
y = data.y.tolist()
y = list(flatten(y))
count_y = collections.Counter(y)
print(count_y)

In [ ]:
# loading node feature matrix and node labels
x = data.x.to(device)
y = data.y.squeeze().to(device)

### Model

In [ ]:
from torch.nn import Linear 
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 heads):
        super(GAT, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(dataset.num_features, hidden_channels,
                                  heads))
        for _ in range(num_layers - 2):
            self.convs.append(
                GATConv(heads * hidden_channels, hidden_channels, heads))
        self.convs.append(
            GATConv(heads * hidden_channels, out_channels, heads, concat=False))

        self.skips = torch.nn.ModuleList()
        self.skips.append(Linear(dataset.num_features, hidden_channels * heads))
        for _ in range(num_layers - 2):
            self.skips.append(
                Linear(hidden_channels * heads, hidden_channels * heads))
        self.skips.append(Linear(hidden_channels * heads, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for skip in self.skips:
            skip.reset_parameters()

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            x = x + self.skips[i](x_target)
            if i != self.num_layers - 1:
                x = F.elu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return x.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in test_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                x = x + self.skips[i](x_target)

                if i != self.num_layers - 1:
                    x = F.elu(x)
                xs.append(x.cpu())

                pbar.update(batch_size)

            x_all = torch.cat(xs, dim=0)

        pbar.close()

        return x_all


### train
dont run this if using test_loader

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT(dataset.num_features, 128, dataset.num_classes, num_layers=3, heads=2)
model = model.to(device)

In [20]:
def train(epoch):
    model.train()

    pbar = tqdm(total=train_idx.size(0))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out = model(x[n_id], adjs)
        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / train_idx.size(0)

    return loss, approx_acc


In [21]:
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

for epoch in range(1, 21):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')


Run 01:



Epoch 01:   0%|          | 512/196615 [00:11<1:11:07, 45.95it/s]

KeyboardInterrupt: ignored

### saving
dont run this if using test_loader

In [22]:
# saving model in mydrive
import random
MODEL_PATH = '/content/drive/MyDrive/MLNS/final_project/models/model.pt'
MODEL_PATH = MODEL_PATH[:-3] + str(random.randint(0,69)) + '.pt'
torch.save(model, MODEL_PATH)
print(f'Model Saved at {MODEL_PATH}')

Model Saved at /content/drive/MyDrive/MLNS/final_project/models/model38.pt


## test 
start from here if using test_loader

In [23]:
# loading the saved model
model = torch.load(MODEL_PATH, map_location=torch.device(device))
model = model.to(device)

In [24]:
@torch.no_grad()
def test():
    model.eval()

    out = model.inference(x)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True) 

    test_acc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return test_acc



In [25]:
test_acc = test()


Evaluating:   0%|          | 0/7347087 [00:00<?, ?it/s]

NameError: ignored

# References



1. [Graph Attention Networks](https://arxiv.org/abs/1710.10903)

2. https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/conv/gat_conv.html

3. [How Attentive are Graph Attention Networks?](https://arxiv.org/abs/2105.14491)


